In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import matplotlib.pyplot as plt

# ----- Device Setup -----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----- Image Loader -----
def load_image(img_path, max_size=512):
    image = Image.open(img_path).convert('RGB')
    size = min(max(image.size), max_size)
    
    transform = transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    
    image = transform(image).unsqueeze(0)
    return image.to(device)

# ----- Image Post-processing -----
def im_convert(tensor):
    image = tensor.clone().detach().cpu().squeeze(0)
    image = image * torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    image = image + torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    image = image.clamp(0, 1)
    return transforms.ToPILImage()(image)

# ----- Load Content & Style Images -----
content_path = r"C:\Users\ghwns\HJ_git\CV-Projects\dl-chapter-notebooks\Chapter 9\CAT.jpg"
style_path = r"C:\Users\ghwns\HJ_git\CV-Projects\dl-chapter-notebooks\Chapter 9\Vincent_Van_Gogh_Style.jpg"
content = load_image(content_path)
style = load_image(style_path)

# ----- Load Pretrained VGG19 Model -----
vgg = models.vgg19(pretrained=True).features.to(device).eval()

# ----- Feature Extraction -----
def get_features(image, model, layers=None):
    if layers is None:
        layers = {
            '0': 'conv1_1',
            '5': 'conv2_1',
            '10': 'conv3_1',
            '19': 'conv4_1',
            '21': 'conv4_2',  # Content layer
            '28': 'conv5_1'
        }

    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x
    return features

# ----- Gram Matrix for Style Representation -----
def gram_matrix(tensor):
    _, c, h, w = tensor.size()
    tensor = tensor.view(c, h * w)
    gram = torch.mm(tensor, tensor.t())
    return gram

# ----- Initialize Target Image -----
target = content.clone().requires_grad_(True).to(device)

# ----- Extract Features -----
content_features = get_features(content, vgg)
style_features = get_features(style, vgg)
style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

# ----- Define Loss Weights -----
style_weights = {
    'conv1_1': 1.0,
    'conv2_1': 0.8,
    'conv3_1': 0.5,
    'conv4_1': 0.3,
    'conv5_1': 0.1
}
content_weight = 1e4
style_weight = 1e2

# ----- Optimizer -----
optimizer = optim.Adam([target], lr=0.003)

# ----- Training Loop -----
steps = 300
for step in range(steps):
    target_features = get_features(target, vgg)
    
    # Content Loss
    content_loss = torch.mean((target_features['conv4_2'] - content_features['conv4_2']) ** 2)

    # Style Loss
    style_loss = 0
    for layer in style_weights:
        target_feature = target_features[layer]
        target_gram = gram_matrix(target_feature)
        style_gram = style_grams[layer]
        layer_loss = torch.mean((target_gram - style_gram) ** 2)
        style_loss += style_weights[layer] * layer_loss / (target_feature.shape[1] ** 2)

    # Total Loss
    total_loss = content_weight * content_loss + style_weight * style_loss

    optimizer.zero_grad()
    total_loss.backward(retain_graph=True)
    optimizer.step()

    if step % 50 == 0:
        print(f"Step {step}, Total loss: {total_loss.item():.4f}")

# ----- Visualize Output -----
final_img = im_convert(target)
plt.imshow(final_img)
plt.title("Neural Style Transfer Result")
plt.axis('off')
plt.show()

Step 0, Total loss: 65858300.0000
Step 50, Total loss: 35222736.0000
Step 100, Total loss: 23758538.0000
Step 150, Total loss: 18407274.0000
Step 200, Total loss: 15060986.0000
Step 250, Total loss: 12526040.0000
